# Pivot Tables

Hemos visto cómo la abstracción `` GroupBy `` nos permite explorar las relaciones dentro de un conjunto de datos.

Una *pivot table* o tabla dinámica es una operación similar que se ve comúnmente en hojas de cálculo y otros programas que operan con datos tabulares.
La tabla dinámica toma datos simples de columnas como entrada y agrupa las entradas en una tabla bidimensional que proporciona un resumen multidimensional de los datos.

La diferencia entre tablas dinámicas y `` GroupBy `` a veces puede causar confusión; pero básicamente lo podemos entender como que las tablas dinámicas son una versión multidimensional de la agregación `` GroupBy ``.

Es decir, divide-aplica-combina, pero tanto la división como la combinación ocurren no en un índice unidimensional, sino en una cuadrícula bidimensional.

En este notebook, utilizaremos los datos que ya hemos visto de los pasajeros del Titanic para desarrollar los conceptos. Datos disponibles a través de la librería Seaborn.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [2]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Este dataset contiene una gran cantidad de información sobre cada pasajero, incluyendo el sexo, la edad, la clase, la tarifa pagada y mucho más.

## Pivot Tables a mano

Para comenzar a aprender más sobre estos datos, podríamos comenzar agrupando según el género, si ha sobrevivido o no, o alguna combinación de los mismos.

En base a lo que ya hemos aprendido, podríamos tener la tentación de aplicar una operación de `` GroupBy ``; por ejemplo, veamos la tasa de supervivencia por género:

In [3]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


Esto inmediatamente nos da una idea: en general, tres de cada cuatro mujeres a bordo sobrevivieron, mientras que solo uno de cada cinco hombres lo hizo.

Esto es útil, pero nos gustaría ir un paso más allá y estudiar la supervivencia por sexo y, por ejemplo, clase.
Usando el `` GroupBy ``, podríamos proceder usando algo como:
Agrupamos por clase y género, *seleccionamos* 'survival, * aplicamos * la agregación de la media, * combinamos * los grupos resultantes, y luego * desapilamos * el índice jerárquico para revelar la multidimensionalidad oculta. En codigo:

In [4]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Esto nos da una idea más precisa de cómo el género y la clase afectaron la supervivencia, pero el código comienza a volverse un tanto confuso.

Si bien cada paso que damos tiene sentido dadas las herramientas que hemos visto, la larga cadena de código no es particularmente fácil de leer o usar.

Este `` GroupBy `` bidimensional es lo suficientemente común como para que Pandas incluya algo que nos facilite su tratamiento, como son las `` pivot_table ``, diseñadas especialmente para este tipo de agregación multidimensional.

## Sintaxis de Pivot Table

A continuación, se recoge el equivalente a la operación anteriore mediante ``Pivot Table``, la cual se puede obtener directamente con el método de los DataFrames `` pivot_table``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Esto es mcho más legible que el enfoque de `` group by `` y produce el mismo resultado.

Como era de esperar de un crucero transatlántico de principios del siglo XX, el gradiente de supervivencia favorece tanto a las mujeres como a las clases altas.
Las mujeres de primera clase sobrevivieron con un porcentaje muy elevado, mientras que solo uno de cada diez hombres de tercera clase lo hizo.

### Pivot Tables multinivel

Al igual que en el `` GroupBy ``, la agrupación en tablas dinámicas se puede especificar con múltiples niveles y mediante varias opciones.

Por ejemplo, podríamos estar interesados en considerar la edad como una tercera dimensión. Pero meter un dato de una variable con tanta variación podría desagregarnos tanto los datos que no sacaríamos nada en concreto. Por ello, resultaría más interesante realizar cierta agrupación de edades, como hacer una división entre menores de edad (<18), adultos (>18 y <65) y jubilados (>65), lo cual lo podemos conseguir gracias a la función `` pd.cut ``:

In [19]:
age = pd.cut(titanic['age'], [0, 18, 65, int(max(titanic['age']))])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 65]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 65]  0.380435  0.073171  0.135000
       (65, 80]  0.250000  0.000000  0.000000

También podríamos aplicar la misma estrategia cuando trabajamos con las columnas. Por ejemplo, podríamos agregar información sobre la tarifa pagada usando `` pd.qcut `` para calcular automáticamente los cuantiles:

In [20]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 65]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 65]              0.0  0.102041  0.126437          0.397727   
       (65, 80]              NaN  0.000000  0.000000          0.250000   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 65]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 65]  0.030303  0.192308  
       (65, 80]       NaN       NaN

El resultado es una agregación de cuatro dimensiones con índices jerárquicos, que se muestra en una cuadrícula que demuestra la relación entre los valores.

### Opciones adicionales de Pivot Table

La llamada completa del método `` pivot_table `` de `` DataFrame `` es la siguiente:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

Ya hemos visto ejemplos de los tres primeros argumentos; por lo que ahora echaremos un vistazo rápido a los restantes.
Dos de las opciones, `` fill_value `` y `` dropna ``, tienen que ver con datos faltantes y son bastante sencillas; por lo que no mostraremos ejemplos de ellos aquí.

Por otra parte, la palabra clave ``aggfunc`` controla qué tipo de agregación se aplica, que por defecto será la media.
Al igual que con el `` GroupBy ``, la especificación de agregación puede ser un string que represente una de varias opciones comunes (por ejemplo, ``sum``, ``mean``, ``count``, ``min``, ``max``...) o una función que implemente una agregación (por ejemplo, `` np.sum ()`` , `` min ()``, `` sum ()``...).

Además, se puede especificar como un diccionario que asigne una columna a cualquiera de las opciones deseadas anteriores:

In [22]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Date cuenta que también aquí que hemos omitido la palabra clave `` values ``, ya que al especificar un mapeo para "aggfunc", este parámetro se determina automáticamente.

También podríamos calcular en base a una combinación de ejes:

In [25]:
titanic.pivot_table(index=['sex', 'embarked'], columns=['class', 'alone'],
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                                               \
class                 First                 Second                 Third   
alone                 False       True       False      True       False   
sex    embarked                                                            
female C         112.003000  120.692128  30.217500  12.895850  16.367589   
       Q          90.000000         NaN        NaN  12.350000  18.284375   
       S         102.443874   90.728571  27.621795  13.960714  23.898852   
male   C         119.409874   72.163222  32.896533  14.208325  13.103770   
       Q          90.000000         NaN        NaN  12.350000  21.815000   
       S          92.783489   32.267146  30.441667  14.213433  24.354565   

                          survived                                 
class                        First       Second       Third        
alone               True     False True   False True  False True   
sex    embarked                                                    
female C         8.673340     25.0  17.0    5.0   2.0  11.0   4.0  
       Q         7.755340      1.0   NaN    NaN   2.0   5.0  19.0  
       S         8.561113     32.0  14.0   36.0  25.0  19.0  14.0  
male   C         8.215409      9.0   8.0    1.0   1.0   5.0   5.0  
       Q         8.513648      0.0   NaN    NaN   0.0   1.0   2.0  
       S         9.861668     11.0  17.0    9.0   6.0   9.0  25.0

A veces es útil calcular los totales de cada agrupación. Esto se puede hacer mediante el parámetro `` margins ``:

In [26]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


Aquí, esto nos da automáticamente información sobre la tasa de supervivencia por género o por clase sin tener en cuenta el otro eje, gracias a los totales. Además, en el registro correspondiente a los dos ``All``, el resultado se corresponde con el total sin tener en cuenta ninguna división, es decir, nos dice que el porcentaje de supervivientes total fue del 38%.

La etiqueta del "margen" que hemos añadido se puede especificar con el parámetro `` margins_name``, que por defecto es `` All ``.

## Ejemplo: Cumpleaños

Veamos ahora un ejemplo más interesante, echemos un vistazo a los datos disponibles sobre nacimientos en los Estados Unidos, proporcionados por los Centros para el Control de Enfermedades (CDC).

Estos datos se pueden encontrar en https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

In [28]:
births = pd.read_csv('births.csv')

Al observar los datos, vemos que es relativamente simple: contiene la cantidad de nacimientos agrupados por fecha y sexo:

In [29]:
births.head()

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


Podemos comenzar a comprender estos datos un poco más usando una pivot table.

Por ejemplo, agreguemos una columna de décadas y echemos un vistazo a los nacimientos de hombres y mujeres en función de la década:

In [ ]:
births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')

gender,F,M
decade,,
1960,1753634,1846572
1970,16263075,17121550
1980,18310351,19243452
1990,19479454,20420553
2000,18229309,19106428


A simple vista, vemos que los nacimientos masculinos superan en número a los nacimientos femeninos en cada década.

## Ejercicio

1. ¿Están limpios los datos? Detecta si tenemos nulos u outliers
2. Elimina los nulos y los outliers, y repite el cálculo anterior, ¿varía mucho?
3. Repite el paso 2 pero en lugar de eliminar lo nulos sustitúyelos por la mediana de cada columna. Después, elimina los outliers con el método del rango intercuartílico
4. Repite el cálculo de nuevo pero sustituyendo los nulos por el valor anterior y los outliers por el máximo o el mínimo del resto de datos de su columna, según corresponda.

In [ ]:
# 1. 


In [ ]:
# 2. 


In [ ]:
# 3. 


In [ ]:
# 4. 


## Ejercicio 2

Lee nuestro más que conocido fichero con datos de jugadores de fútbol ("FIFA20.csv") y construye las siguientes pivot table:

1. Suma de salarios de jugadores en base a su team_position, su preferred_foot y si es influencer o no
2. Media de potential y mínimo de overall en función del año de dob (no la fecha completa), el club y el value_eur agrupado en los buckets [0, 100 000), [100 000, 1 000 000), [1 000 000, max).

In [32]:
# 1. 
import pandas as pd

df = pd.read_csv("FIFA20.csv")
df

,short_name,long_name,dob,club,overall,potential,value_eur,wage_eur,team_position,preferred_foot,influencer
0,L. Messi,Lionel Andrés Messi Cuccittini,1987-06-24,FC Barcelona,94,94,95500000,565000,RW,Left,True
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,1985-02-05,Juventus,93,93,58500000,405000,LW,Right,True
2,Neymar Jr,Neymar da Silva Santos Junior,1992-02-05,Paris Saint-Germain,92,92,105500000,290000,CAM,Right,True
3,J. Oblak,Jan Oblak,1993-01-07,Atlético Madrid,91,93,77500000,125000,GK,Right,False
4,E. Hazard,Eden Hazard,1991-01-07,Real Madrid,91,91,90000000,470000,LW,Right,True
...,...,...,...,...,...,...,...,...,...,...,...
18273,Shao Shuai,邵帅,1997-03-10,Beijing Renhe FC,48,56,40000,2000,RES,Right,False
18274,Xiao Mingjie,Mingjie Xiao,1997-01-01,Shanghai SIPG FC,48,56,40000,2000,SUB,Right,False
18275,Zhang Wei,张威,2000-05-16,Hebei China Fortune FC,48,56,40000,1000,SUB,Right,False
18276,Wang Haijian,汪海健,2000-08-02,Shanghai Greenland Shenhua FC,48,54,40000,1000,SUB,Right,False
